# 07 Send CSV report

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # emplacement des modules RenPri
from issep import sarsar_admin
from os.path import join
from lecture_ini import config

In [ ]:
def send_report(dteChgEnd, to='g.swinnen@issep.be c.wyard@issep.be'):
    
    # Emplacement où sera créé le rapport CSV
    chemins = config(section='chemins')
    fichier = join(chemins['rapports'], '{0}_DIGIT.csv'.format(dteChgEnd))  # r'/home/cowy/bimestriel_{0}_DIGIT.csv'.format(dteChgEnd)

    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass
    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env et the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)

    # Ouvre la connexion à la DB
    conn = sarsar_admin._create_or_get_db_connection()

    try:
        import psycopg2.extras
        
        # Lit les données en tant que dataframe pandas et les sauve en tant que CSV
#        strSQL = 'SELECT * FROM "bimestriel_{0}_DIGIT";'.format(dteChgEnd)
        strSQL = '''SELECT a.* FROM "bimestriel_{0}_DIGIT" a 
                    INNER JOIN (SELECT codecarto, max(ordre_de_priorite) AS priorite_max FROM "bimestriel_{0}_DIGIT" GROUP BY codecarto) b 
                    ON a.codecarto = b.codecarto AND a.ordre_de_priorite = b.priorite_max 
                    ORDER BY a.ordre_de_priorite DESC, a.codecarto ASC;'''.format(dteChgEnd)
        
        rapport = pd.read_sql_query(strSQL,conn)
        rapport.to_csv(path_or_buf=fichier, sep=';', na_rep='NA', decimal=',', header=True, index=False, mode='w')
        
        # Envoi du rapport par courriel
        subj = 'SARSAR: rapport de détection de changements'
        message = 'Ci-joint le rapport au format CSV, directement ouvrable dans Excel'
        files = fichier

        courriel = config(section='courriel')
        commande = 'sendemail -f {0} -t {1} -u "{2}" -s {3} -m "{4}" -a {5} -v -o message-charset={6}'.format(courriel['from'], to, subj, courriel['server'], message, files, courriel['charset'])
        print(commande)

        os.system(commande)
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
            
    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()